In [1]:
# your other imports here ...
import sys, os
import pandas as pd
import matplotlib.pyplot as plt

# TODO: replace with your path/to/ninarow
ninarowdir = os.path.dirname(os.getcwd())
modelfitdir = ninarowdir + "/model_fitting/"
# os.listdir(modelfitdir)

# sets the import path to the model-fitting directory
sys.path.insert(0, modelfitdir)
from parsers import *
# from new_fit import *
from utils import *
from utils import show
from tqdm import tqdm
import random

# WARNING: %load_ext autoreload and %autoreload 2 may interfere with 
# the Multi-threading processes!
%load_ext autoreload
%autoreload 2

In [4]:
df = pd.read_csv("../data/data.csv")
df["response_time"] = 1
df = df[:100]
df.head()
make_splits(df, 5, "../data");

Saving split1 to ../data/1.csv
Saving split2 to ../data/2.csv
Saving split3 to ../data/3.csv
Saving split4 to ../data/4.csv
Saving split5 to ../data/5.csv


In [5]:
data_path = "../data"
output_path = "../data/out"
n_splits = 5
fold_number = 1
threads = 1
random_sample = False
verbose = True

print(f"Building output directory at {output_path}")
os.makedirs(output_path, exist_ok = True)

# first, we have to check to see if all the splits are there ...
assert np.all([f"{i + 1}.csv" in os.listdir(data_path) for i in range(n_splits)])
print("Detected splits in this directory. Loading splits ...")

# then we read them in
splits = [pd.read_csv(f"{data_path}/{i + 1}.csv") for i in range(n_splits)]

# we convert every row of our CSV to a "CSVMove object" using df_to_CSVMove - we do so for all the splits
# CSVMove is a class that is defined in the parsers.py file 
fold_data = [[csvmove for csvmove in df_to_CSVMove(split, warn = True)] for split in splits]

Building output directory at ../data/out
Detected splits in this directory. Loading splits ...


In [4]:
fold_data[0]

[4096	0	White	16	1.0	1	1	3061b542-0eec-4ef3-bdaa-6ced5a0698c6,
 4840226848	7340032	White	524288	1.0	1	1	6cb67fda-9801-4bca-b23c-6e01c83493b5,
 4379902208	14696448	White	4096	1.0	1	1	bbc01452-976f-4204-b1f4-18cec480bd22,
 4096	0	White	4194304	1.0	1	1	30b9e798-230f-4f87-955c-3dd8466b1508,
 12288	4194304	White	2097152	1.0	1	1	b34c0a85-0944-4dbd-93c1-764afe36e114,
 4312793344	14680064	White	16384	1.0	1	1	023c0811-614f-4024-add2-8e36db9c509f,
 8396808	3145728	White	524288	1.0	1	1	d9a0d256-38ee-4b77-bb2a-06cc2b1b2dcf,
 8396800	2097152	White	1048576	1.0	1	1	6d5e566f-d97f-4e79-91ed-06af8b2cb808,
 4096	0	White	2147483648	1.0	1	1	3ae83c4c-c02b-4d82-8fd5-ab9f12e1300b,
 1073741824	0	White	2097152	1.0	1	1	cfb54d41-9b84-4828-b3b1-66e4a00cf27d]

In [5]:
import model
from time import time
# from utils import *

random.seed(10)
model.initialize_thread_pool(1, manual_seed = 10)
model_fitter =  model.Fitter(model.DefaultModel(), threads = 1, verbose = True)

tick = time()
# q = model.cross_validate(model.DefaultModel(), splits, leave_out_idx = 4, threads = 1)
model_fitter.fit(pd.concat([splits[0], splits[2], splits[3], splits[4]]))
print(time() - tick)

Setting manual seed 10 for single-thread
Thread 0: Base Seed 10, Seed: 10, Random Number: 601088376405717203

[Preprocessing] Initial log-likelihood estimation


  0%|          | 0/10 [00:05<?, ?it/s]


KeyboardInterrupt: 

In [ ]:
import model_fit
from time import time
from parsers import *

class DotDict(dict):
    """A dictionary that supports dot notation access."""
    def __getattr__(self, name):
        if name in self:
            return self[name]
        raise AttributeError(f"'{self.__class__.__name__}' object has no attribute '{name}'")
    
    def __setattr__(self, name, value):
        self[name] = value


args = DotDict({"random_sample": random_sample, 
    "verbose": verbose, 
    "threads": threads})

random.seed(10)
model_fit.initialize_thread_pool(1, manual_seed = 10)
model_fitter =  model_fit.ModelFitter(model_fit.DefaultModel(), args)

tick = time()
model_fitter.fit_model(fold_data[0] + fold_data[2] + fold_data[3] + fold_data[4])
print(time() - tick)

Setting manual seed 10 for single-thread
Thread 0: Base Seed 10, Seed: 10, Random Number: 601088376405717203

Beginning model fit pre-processing: log-likelihood estimation


  0%|          | 0/10 [00:00<?, ?it/s]

First integer of RNG state: 2699640232, Stopping threshold: 10000, Pruning threshold: 2, Gamma: 0.02, Lapse rate: 0.05, Opp scale: 1, Exploration constant: 1, Center weight: 0.8, Noise enabled: 1, Search in progress: 1
First integer of RNG state: 2457535207, Stopping threshold: 10000, Pruning threshold: 2, Gamma: 0.02, Lapse rate: 0.05, Opp scale: 1, Exploration constant: 1, Center weight: 0.8, Noise enabled: 1, Search in progress: 0
First integer of RNG state: 215589996, Stopping threshold: 10000, Pruning threshold: 2, Gamma: 0.02, Lapse rate: 0.05, Opp scale: 1, Exploration constant: 1, Center weight: 0.8, Noise enabled: 1, Search in progress: 1
First integer of RNG state: 450611884, Stopping threshold: 10000, Pruning threshold: 2, Gamma: 0.02, Lapse rate: 0.05, Opp scale: 1, Exploration constant: 1, Center weight: 0.8, Noise enabled: 1, Search in progress: 0
First integer of RNG state: 408086476, Stopping threshold: 10000, Pruning threshold: 2, Gamma: 0.02, Lapse rate: 0.05, Opp sca

  0%|          | 0/10 [00:03<?, ?it/s]

First integer of RNG state: 2485423373, Stopping threshold: 10000, Pruning threshold: 2, Gamma: 0.02, Lapse rate: 0.05, Opp scale: 1, Exploration constant: 1, Center weight: 0.8, Noise enabled: 1, Search in progress: 0
First integer of RNG state: 3805881228, Stopping threshold: 10000, Pruning threshold: 2, Gamma: 0.02, Lapse rate: 0.05, Opp scale: 1, Exploration constant: 1, Center weight: 0.8, Noise enabled: 1, Search in progress: 1
First integer of RNG state: 2204615923, Stopping threshold: 10000, Pruning threshold: 2, Gamma: 0.02, Lapse rate: 0.05, Opp scale: 1, Exploration constant: 1, Center weight: 0.8, Noise enabled: 1, Search in progress: 0
First integer of RNG state: 4073813386, Stopping threshold: 10000, Pruning threshold: 2, Gamma: 0.02, Lapse rate: 0.05, Opp scale: 1, Exploration constant: 1, Center weight: 0.8, Noise enabled: 1, Search in progress: 1
First integer of RNG state: 1365086080, Stopping threshold: 10000, Pruning threshold: 2, Gamma: 0.02, Lapse rate: 0.05, Opp 

First integer of RNG state: 3591741175, Stopping threshold: 10000, Pruning threshold: 2, Gamma: 0.02, Lapse rate: 0.05, Opp scale: 1, Exploration constant: 1, Center weight: 0.8, Noise enabled: 1, Search in progress: 1
First integer of RNG state: 4091201681, Stopping threshold: 10000, Pruning threshold: 2, Gamma: 0.02, Lapse rate: 0.05, Opp scale: 1, Exploration constant: 1, Center weight: 0.8, Noise enabled: 1, Search in progress: 0
First integer of RNG state: 3797744536, Stopping threshold: 10000, Pruning threshold: 2, Gamma: 0.02, Lapse rate: 0.05, Opp scale: 1, Exploration constant: 1, Center weight: 0.8, Noise enabled: 1, Search in progress: 1
First integer of RNG state: 3013362023, Stopping threshold: 10000, Pruning threshold: 2, Gamma: 0.02, Lapse rate: 0.05, Opp scale: 1, Exploration constant: 1, Center weight: 0.8, Noise enabled: 1, Search in progress: 0
First integer of RNG state: 2202315409, Stopping threshold: 10000, Pruning threshold: 2, Gamma: 0.02, Lapse rate: 0.05, Opp 

KeyboardInterrupt: 

First integer of RNG state: 3708638709, Stopping threshold: 10000, Pruning threshold: 2, Gamma: 0.02, Lapse rate: 0.05, Opp scale: 1, Exploration constant: 1, Center weight: 0.8, Noise enabled: 1, Search in progress: 0
First integer of RNG state: 2065134385, Stopping threshold: 10000, Pruning threshold: 2, Gamma: 0.02, Lapse rate: 0.05, Opp scale: 1, Exploration constant: 1, Center weight: 0.8, Noise enabled: 1, Search in progress: 1


First integer of RNG state: 3728963243, Stopping threshold: 10000, Pruning threshold: 2, Gamma: 0.02, Lapse rate: 0.05, Opp scale: 1, Exploration constant: 1, Center weight: 0.8, Noise enabled: 1, Search in progress: 0
First integer of RNG state: 2623003206, Stopping threshold: 10000, Pruning threshold: 2, Gamma: 0.02, Lapse rate: 0.05, Opp scale: 1, Exploration constant: 1, Center weight: 0.8, Noise enabled: 1, Search in progress: 1
First integer of RNG state: 2537528872, Stopping threshold: 10000, Pruning threshold: 2, Gamma: 0.02, Lapse rate: 0.05, Opp scale: 1, Exploration constant: 1, Center weight: 0.8, Noise enabled: 1, Search in progress: 0
First integer of RNG state: 3270691817, Stopping threshold: 10000, Pruning threshold: 2, Gamma: 0.02, Lapse rate: 0.05, Opp scale: 1, Exploration constant: 1, Center weight: 0.8, Noise enabled: 1, Search in progress: 1
First integer of RNG state: 989524301, Stopping threshold: 10000, Pruning threshold: 2, Gamma: 0.02, Lapse rate: 0.05, Opp s